In [1]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim_models
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
%config InlineBackend.figure_formats = ['retina']
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
import matplotlib.pyplot as plt
from top2vec import Top2Vec
from nltk.corpus import stopwords
from gensim.models import TfidfModel
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.datasets import load_iris

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\numba\np\arraymath.py:3845: DeprecationWarning: `np.MachAr` is deprecated (NumPy 1.22).
  @overload(np.MachAr)


In [2]:
# import ipynb
# from ipynb.fs.full.topic_models import split_data, lda_vecs

In [3]:
training_data = pd.read_csv("training_data/training_data.csv")
test_data = pd.read_csv("test_data/test_data.csv")

In [4]:
test_data.target.value_counts()

0    179
1    171
Name: target, dtype: int64

In [5]:
################################################### lda ##############################################################################

In [6]:
lda_train_vecs = pd.read_csv("vecs/lda_vecs.csv")

In [7]:
lda_model = gensim.models.ldamodel.LdaModel.load("models/lda_train.model")

In [8]:
test_docs = test_data.allegation_desc

def lemmatization(descs, allowed_pos_tags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    final_text = []
    for desc in descs:
        doc = nlp(desc)
        new_text = " ".join([token.lemma_ for token in doc if token.pos_ in allowed_pos_tags])
        final_text.append(new_text)
    return (final_text)

lemmatized_texts = lemmatization(test_docs)

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return list(bigram[doc] for doc in texts)

def make_trigrams(texts):
    return list(trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_words)

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

test_corpus = [id2word.doc2bow(text) for text in texts]

tdidf = TfidfModel(test_corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tdif = []

for i in range(0, len(test_corpus)):
    bow = test_corpus[i]
    low_value_words = []
    tdif_ids = [id for id, value in tdidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tdidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tdif
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tdif = [id for id in bow_ids if id not in tdif_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tdif]
    test_corpus[i] = new_bow

def get_test_vecs():
    lda_test_vecs = []
    for i in range(len(test_docs)):
        top_topics = lda_model.get_document_topics(test_corpus[i], minimum_probability=0.0)
        topic_vec = [top_topics[i][1] for i in range(10)]
        topic_vec.extend([len(test_docs.iloc[i])]) # length review
        lda_test_vecs.append(topic_vec)
    return lda_test_vecs

In [9]:
lda_test_vecs = get_test_vecs()

In [10]:
################################################### top2vec ##############################################################################

In [11]:
top_2_vec_model_train = Top2Vec.load("models/noso")
top_train_vecs = top_2_vec_model_train.document_vectors

In [12]:
def convert_test_data_to_list(data):
    test_docs = [x for x in data["allegation_desc"]]
    return test_docs

In [14]:
test_docs = convert_test_data_to_list(test_data)

top_2_vec_model_test = Top2Vec(test_docs, embedding_model_path="models/noso")

top_test_vecs = top_2_vec_model_test.document_vectors

2023-01-31 15:30:47,910 - top2vec - INFO - Pre-processing documents for training
2023-01-31 15:30:47,951 - top2vec - INFO - Creating joint document/word embedding
2023-01-31 15:30:48,737 - top2vec - INFO - Creating lower dimension embedding of documents
2023-01-31 15:30:53,249 - top2vec - INFO - Finding dense areas of documents
2023-01-31 15:30:53,259 - top2vec - INFO - Finding topics


In [15]:
############################################################# regress #################################################################

In [16]:
X_test = np.array(top_test_vecs)
y_test = np.array(test_data.target)

In [17]:
# X_test = np.pad(X_test, pad_width=((0,350),(0,0)), mode='constant')
X_test.shape

(350, 300)

In [18]:
X = np.array(top_train_vecs)
X.shape

(700, 300)

In [19]:
y = np.array(training_data.target)

In [20]:
kf = KFold(5, shuffle=True, random_state=48)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

for train_ind, val_ind in kf.split(X, y):
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        fit_intercept=True,
    ).fit(X_train_scale, y_train)

    test_data["lr_scores"] = lr.predict(scaler.transform(X_test))
    test_data["lr_scores_prob_1"] = lr.predict_proba(scaler.transform(X_test))[:, 1]

    y_pred = lr.predict(scaler.transform(X_val_scale))
    cv_lr_f1.append(f1_score(y_val, y_pred, average='weighted'))
    
    # Logistic Regression Mini-Batch SGD
    sgd = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)

    test_data["sgd_scores"] = sgd.predict(scaler.transform(X_test))
    test_data["sgd_scores_prob_1"] = sgd.predict_proba(scaler.transform(X_test))[:, 1]

    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average="weighted"))
    
    # SGD Modified Huber
    sgd_huber = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20,
        loss='modified_huber',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)

    test_data["sgd_huber_scores"] = sgd_huber.predict(scaler.transform(X_test))
    test_data["sgd_huber_scores_prob_1"] = sgd_huber.predict_proba(scaler.transform(X_test))[:, 1]
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average="weighted"))

print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\opt

Logistic Regression Val f1: 0.621 +- 0.013
Logisitic Regression SGD Val f1: 0.579 +- 0.006
SVM Huber Val f1: 0.367 +- 0.073


c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


In [21]:
# test_pred = test_data[~((test_data.target == "0"))]
# test_pred
test_data

,Unnamed: 0,id,allegation_desc,meta_agency,meta_tracking_id,label,target,lr_scores,lr_scores_prob_1,sgd_scores,sgd_scores_prob_1,sgd_huber_scores,sgd_huber_scores_prob_1
0,613,16272,complainant documented the accused officers di...,new-orleans-pd,2016-0706-p,inadequate description,0,0,0.000062,0,3.577752e-113,0,0.250132
1,715,16740,complaint: complainant stated police came to a...,new-orleans-pd,2017-0385-r,evidence,0,1,0.913350,1,1.000000e+00,0,0.251412
2,93,13924,reported after scheduled time for duty.,new-orleans-so,D-015-19,internal misconduct/administrative infractions,1,0,0.010896,0,4.174592e-47,0,0.246417
3,962,17848,officers were accused of not documenting evide...,new-orleans-pd,2020-0078-r,evidence,0,1,0.724844,1,1.000000e+00,0,0.250613
4,808,17155,complainant accused officer of failing to veri...,new-orleans-pd,2018-0236-p,evidence,0,1,0.999789,1,1.000000e+00,0,0.263790
...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,565,16048,complainant stated that she was falsely arrest...,new-orleans-pd,2016-0462.p,arrests/stops/searches,0,0,0.000006,0,6.674609e-156,0,0.265821
346,666,16507,complainant stated that an unknown nopd office...,new-orleans-pd,2017-0128-p,traffic,0,0,0.000017,0,2.041634e-140,0,0.265987
347,629,16344,the officer illegally search a subject’s backp...,new-orleans-pd,2016-0786.r,arrests/stops/searches,0,0,0.405793,0,2.376089e-08,0,0.195111
348,355,15103,officer allegedly screamed and yelled at the c...,new-orleans-pd,2014-0771-c,harm to the general public safety,1,1,0.991661,1,1.000000e+00,0,0.264261
